# Jetpack Compose - Flow - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu `Flow`.

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExbHdhamV6bDFidmNvMmVmc250eWxuYTJ6aWMxOGljZGZocmxxczRuOSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/xuvbEBtBvw4PMnDOwS/giphy.gif" width="200" />

`Flow` umożliwia asynchroniczne przetwarzanie sekwencji wartości, które mogą być emitowane asynchronicznie i odbierane w reaktywny sposób. Dzięki temu można implementować strumienie danych, obsługiwać opóźnienia, operacje na wielu wartościach oraz obsługiwać błędy i anulowanie strumienia.

Strumienie danych (ang. *data streams*) to sekwencje wartości lub zdarzeń, które są emitowane w określonym czasie lub w odpowiedzi na różne zdarzenia. Strumienie danych mogą być dynamicznie aktualizowane i przetwarzane.

Główne cechy strumieni danych to:
- Emitowanie wartości: Strumienie danych mogą emitować wartości w czasie, jedną po drugiej. Emitowanie wartości może być synchroniczne lub asynchroniczne. 
- Reaktywne odbieranie: Odbiorcy strumienia mogą reagować na emitowane wartości, podejmując odpowiednie działania. Odbieranie może odbywać się w czasie rzeczywistym lub w reakcji na określone zdarzenia.
- Przetwarzanie strumienia: Strumienie danych mogą być przetwarzane i transformowane za pomocą różnych operacji, takich jak filtrowanie, mapowanie, łączenie, grupowanie itp.
- Obsługa błędów i anulowanie: Strumienie danych mogą obsługiwać sytuacje błędne i obsługiwać anulowanie strumienia w dowolnym momencie.

W naszej aplikacji utworzymy listę danych (*dummy data*). Będziemy posiadać jedno pole `Text`, w którym wyświetlimy każdy kolejny element listy co 0,5 sekundy.

Dodajmy wymagane zależności

```kotlin
    implementation "androidx.lifecycle:lifecycle-viewmodel-compose:2.6.1"
    implementation "androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1"
    implementation "androidx.lifecycle:lifecycle-runtime-compose:2.6.1"
```

Rozpocznijmy od dodania danych.

In [ ]:
object DataProvider {
    val data: List<String> = listOf(
        "pierwszy",
        "drugi",
        "trzeci",
        "czwarty",
        "piąty",
        "szósty",
        "siódmy",
        "ósmy",
        "dziewiąty",
        "dziesiąty",
        "jedenasty",
        "dwunasty",
        "trzynasty",
        "czternasty",
        "piętnasty",
        "szesnasty",
        "siedemnasty",
        "osiemnasty",
        "dziewiętnasty",
        "dwudziesty",
    )
}

Dodajmy `ViewModel` i utwórzmy nasz `Flow`

In [ ]:
class WordsViewModel : ViewModel() {
    val wordsFlow = flow{
        var index = 0
        while (index < DataProvider.data.size){
            emit(DataProvider.data[index])
            delay(500L)
            index++
        }
    }
}

- `val wordsFlow = flow { ... }` -  Strumień danych `wordsFlow` tworzymy za pomocą funkcji `flow`. flow jest funkcją z biblioteki Kotlin Coroutines, która umożliwia tworzenie strumieni asynchronicznych.
- `var index = 0` - Służy do śledzenia aktualnego indeksu danych.
- `while (index < DataProvider.data.size) { ... }` - Pętla, która działa dopóki `index` jest mniejszy niż rozmiar danych dostarczonych przez `DataProvider`.
- `emit(DataProvider.data[index])` - Wywołanie funkcji `emit` wewnątrz strumienia, która emituje aktualny element danych na strumień.
- `delay(500L)` - Opóźnienie o 500 milisekund przy użyciu funkcji `delay` z biblioteki `Coroutines`.

`WordsViewModel` dostarcza strumień danych `wordsFlow`, który emituje słowa w określonym czasie, z danych dostarczonych przez DataProvider w interwałach co pół sekundy. Dzięki temu strumieniowi można obserwować i reagować na emitowane wyrazy w reaktywny sposób

Kiedy funkcja `emit` jest wywoływana, wartość podana jako argument jest wysyłana do strumienia danych. Ta wartość staje się dostępna dla wszystkich odbiorców, którzy subskrybują ten strumień. W momencie wywołania `emit`, strumień przekazuje tę wartość do aktualnie aktywnych odbiorców. Działa w sposób **asynchroniczny**, co oznacza, że nie blokuje bieżącego wątku podczas wysyłania wartości do strumienia. Dzięki temu możliwe jest kontynuowanie przetwarzania i wykonywania innych operacji jednocześnie.

Przejdźmy do funkcji `@Composable` renderującej nasz interfejs użytkownika, wpierw tworzymy instancję `WordViewModel`

In [ ]:
val viewModel: WordsViewModel = viewModel()

Następnie tworzymy zmienną `word`, która jest typu `State<String>`, używając metody `collectAsStateWithLifecycle` na polu `wordsFlow` z `viewModel`.

In [ ]:
val word = viewModel.wordsFlow.collectAsStateWithLifecycle("start")

`collectAsStateWithLifecycle` zbiera wartości emitowane przez strumień `wordsFlow` i zwraca je jako stan. Stan ten jest automatycznie aktualizowany w zależności od cyklu życia widoku (`viewlifecycle`). Jeśli strumień jest aktywny i emituje wartości, `word` zostanie automatycznie zaktualizowane najnowszą emitowaną wartością.

Wartość `"start"` przekazana jako argument `collectAsStateWithLifecycle`, zostanie użyta jako początkowa wartość do momentu otrzymania pierwszej emitowanej wartości.

`Flow` to jhest tzw. **zimny strumień** (ang. *cold stream*). Oznacza to, że strumień `Flow` nie rozpocznie emisji wartości dopóki nie pojawi się odbiorca, który **zasubskrybuje** ten strumień. W przypadku zimnego strumienia, emisja danych jest inicjowana tylko wtedy, gdy istnieje aktywny odbiorca, który oczekuje na te wartości. Gdy nowy odbiorca subskrybuje strumień, emisja rozpocznie się od początku dla tego konkretnego odbiorcy.

Zaletą zimnych strumieni jest to, że emisja danych jest inicjowana i kontrolowana przez odbiorcę. Pozwala to na bardziej elastyczną obsługę strumieni w zależności od potrzeb, unikając niepotrzebnego generowania i przetwarzania danych, gdy nie ma odbiorców gotowych na ich przyjęcie.

Subskrypcja strumienia `Flow` jest wykonana przy użyciu `collect` (`suspend function`), który umożliwia odbiór i przetwarzanie wartości emitowanych przez strumień.

Aby zasubskrybować strumień, należy wywołać funkcję collect na strumieniu i przekazać blok kodu, który zostanie wykonany dla każdej emitowanej wartości. Przykładowo:

```kotlin
flow.collect { value ->
    // Kod do przetwarzania wartości
}
```

Wewnątrz bloku `collect` można umieścić kod, który będzie przetwarzał każdą emitowaną wartość. Strumień wywoła ten blok kodu dla każdej emitowanej wartości, dostarczając ją jako argument do bloku.

W tym przykładzie wykorzystujemy `collectAsStateWithLifecycle`, `collect` nie jest automatycznie świadome cyklu życia. Oznacza to, że trzeba samodzielnie zarządzać rozpoczęciem i zakończeniem subskrypcji. `collectAsStateWithLifecycle` jest świadome cyklu życia, ponieważ jest zintegrowane z cyklem życia komponentów, takich jak `LifecycleOwner` (podobnie jak `LiveData`). Automatycznie rozpoczyna subskrypcję, gdy komponent jest aktywny, a kończy, gdy jest nieaktywny, co zapewnia bezpieczne zarządzanie subskrypcją w kontekście cyklu życia komponentu.

Mamy do dyspozycji również metodę `collectAsState`, gdy ją wykorzystamy emisja pozostaje aktywna, dopóki nie zostanie ręcznie przerwana, lub zakończona. Nie jest to (zawsze) pożądany efekt, przykładowo jeżeli nasza aplikacja zostanie zminimalizowana, chcemy aby emisja danych została wstrzymana - to oferuje `collectAsStateWithLifecycle`.

Do wartości obiektu `State` dostajemy się poprzez pole `value`

In [ ]:
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            FlowBasicsComposeTheme {

                val viewModel: WordsViewModel = viewModel()

                val word = viewModel.wordsFlow.collectAsStateWithLifecycle("start")

                Surface(
                    modifier = Modifier.fillMaxSize(),
                    color = MaterialTheme.colorScheme.background
                ) {
                    Column(
                        modifier = Modifier.fillMaxSize(),
                        verticalArrangement = Arrangement.Center,
                        horizontalAlignment = Alignment.CenterHorizontally
                    ){
                        Text(
                            text = word.value, // ustawienie wartości w polu Text
                            fontSize = 56.sp,
                            modifier = Modifier.fillMaxWidth(),
                            textAlign = TextAlign.Center
                        )
                    }
                }
            }
        }
    }
}

Możemy przetestować apliakcję

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExbHdhamV6bDFidmNvMmVmc250eWxuYTJ6aWMxOGljZGZocmxxczRuOSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/xuvbEBtBvw4PMnDOwS/giphy.gif" width="200" />